# Apriltag Offset Determination and Calculation

## Imports

In [1]:
import sys
import time
import matplotlib.pyplot as plt
import cv2
import math
import numpy
import scipy.stats as stats
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt

import delivery_driver as dd

sys.path.append("../../deliveryrobot")
from utilities.utilities import *
from sensors.camera.apriltagsensor import *
from sensors.calibration.camera_calibration import *

## Camera Initialization

In [2]:
robot = dd.DeliveryRobot()
sensor = AprilTagSensor(cal_dir)

RuntimeError: Could not initialize camera.  Please see error trace.

In [ ]:
actual_measurements = [0., 0., 0., 0., 0.]
distance = np.array([0.3, 0.6, 0.9, 1.2])

yaw = []
yaw_min = []
yaw_mean = []
yaw_max = []
yaw_stdev = []

meas_idx = 0

In [ ]:
count = 0

yaw_n = []

while count < 50:
    april_img = robot.take_picture(image_dir + "/testing/apriltagsensor_test/")
    measurements={}
    sensor.detect(april_img, measurements)
    #print(f"{measurements['1'][0]}\t{measurements['1'][1]}\t{measurements['1'][2]}")
    yaw_n.append(measurements['1'][2])
    count += 1
    
yaw_n = np.array(yaw_n)
yaw.append(np.mean(yaw_n))
yaw_min.append(np.min(yaw_n))
yaw_mean.append(np.mean(yaw_n))
yaw_max.append(np.max(yaw_n))
yaw_stdev.append(np.std(yaw_n))

# After the loop, create a histogram
plt.figure(figsize=(10, 6))
plt.hist(yaw_n, bins=10, color='blue', alpha=0.7)

# Add titles and labels
plt.title(f'Histogram of Yaw = {actual_measurements[meas_idx]} After 50 Iterations')
plt.xlabel('Yaw Measurement')
plt.ylabel('Frequency')

# Show the plot
plt.savefig(f"psi_{actual_measurements[meas_idx]}_distribution.png")

meas_idx += 1

## Determine MOE for Viability of Measurement Sets

In [ ]:
print(yaw)
print(yaw_min)
print(yaw_mean)
print(yaw_max)
print(yaw_stdev)

In [ ]:
# Python notebook cell to calculate the margin of error based on readings
def calculate_margin_of_error(std_devs, n, confidence_level=0.95):
    # Calculate z-score for the given confidence level
    z_score = stats.norm.ppf((1 + confidence_level) / 2)
    
    # Calculate margin of error
    margin_of_error = z_score * (std_devs / np.sqrt(n))
    return margin_of_error

# Determine MOE
n = 50  # Sample size
margin_of_error = calculate_margin_of_error(yaw_stdev, n)
margin_of_error

## Determine Quadratic Model

In [ ]:
# Function to fit a quadratic model
distance = np.array([0.3,0.6,0.9,1.2])
print(distance)
def fit_quadratic_model(distance, mean_error):
    p = Polynomial.fit(distance, mean_error, 2)
    return p

# Fit the quadratic model
quadratic_model = fit_quadratic_model(distance, yaw_mean)
print("Quadratic Model:", quadratic_model)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import Polynomial

# get predicted error
simulated_distances = np.linspace(distance[0],distance[-1]+1,1000)
predicted_error = quadratic_model(simulated_distances)

# Transform x-values to log(x)
log_x = np.log(distance)

# Perform linear fit on log-transformed x
coefficients = np.polyfit(log_x, yaw_mean, 1)

# Extract the coefficients
a = coefficients[0]  # Slope
b = coefficients[1]  # Intercept

# Logarithmic model: y = a * log(x) + b
y_fit = a * np.log(simulated_distances) + b

# Plot the original data points and the logarithmic fit
plt.scatter(distance, yaw_mean, label="Data Points")
plt.plot(simulated_distances, y_fit, label="Logarithmic Fit", color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

# Output coefficients
print(f"Fitted model: y = {a:.3f} * log(x) + {b:.3f}")

In [ ]:
print(a,b)

## Determine Coefficient of Determination $R^2$

In [ ]:
# Define a function to calculate R-squared for the logarithmic fit
def calculate_r_squared(x, y, a, b):
    # Predicted values from the logarithmic model y = a * log(x) + b
    y_pred = a * np.log(x) + b
    
    # Mean of the actual y-values
    y_mean = np.mean(y)
    
    # Total sum of squares (variance of the data)
    ss_total = np.sum((y - y_mean) ** 2)
    
    # Residual sum of squares (variance not explained by the model)
    ss_residual = np.sum((y - y_pred) ** 2)
    
    # R-squared value
    r_squared = 1 - (ss_residual / ss_total)
    
    return r_squared

# Calculate R-squared for the logarithmic model
r_squared = calculate_r_squared(distance, yaw_mean, a, b)
print(f"R-squared: {r_squared:.4f}")